In [ ]:
import json
from geopy.distance import geodesic

def is_valid_coordinate(coordinate_tuple, direction):

def dms_to_decimal(coordinate_tuple, direction):
    if len(coordinate_tuple) == 1:
        degrees = coordinate_tuple[0]
        minutes = 0
        seconds = 0
    elif len(coordinate_tuple) == 2:
        degrees, minutes = coordinate_tuple
        seconds = 0
    elif len(coordinate_tuple) == 3:
        degrees, minutes, seconds = coordinate_tuple
    else:
        raise ValueError("Invalid coordinate tuple. Must be of length 1, 2, or 3.")
    dd = degrees + (minutes / 60.0) + (seconds / 3600.0)
    if direction in ['S', 'W']:  # South and West are negative
        dd *= -1
    return dd

def generate_bbox_geojson(latitude, longitude, side_length_km=2):
    """
    Generate a GeoJSON bounding box centered around a given coordinate with a specified side length.
    :param lat: Latitude of the center point
    :param lon: Longitude of the center point
    :param side_length_km: Length of the bounding box side in kilometers (default is 2 km)
    :return: GeoJSON object as a dictionary
    """
    if isinstance(latitude, tuple):
        latitude = dms_to_decimal(latitude)
    if isinstance(longitude, tuple):
        longitude = dms_to_decimal(longitude)

    half_side = side_length_km / 2
    
    # Calculate the bounding box coordinates
    top_left = geodesic(kilometers=half_side).destination((latitude, longitude), bearing=315)  # NW
    top_right = geodesic(kilometers=half_side).destination((latitude, longitude), bearing=45)  # NE
    bottom_right = geodesic(kilometers=half_side).destination((latitude, longitude), bearing=135)  # SE
    bottom_left = geodesic(kilometers=half_side).destination((latitude, longitude), bearing=225)  # SW
    
    # GeoJSON format
    geojson = {
        "type": "Feature",
        "geometry": {
            "type": "Polygon",
            "coordinates": [[
                [top_left.longitude, top_left.latitude],
                [top_right.longitude, top_right.latitude],
                [bottom_right.longitude, bottom_right.latitude],
                [bottom_left.longitude, bottom_left.latitude],
                [top_left.longitude, top_left.latitude]  # Closing the polygon
            ]]
        },
        "properties": {}
    }
    
    return json.dumps(geojson, indent=2)

def create_geojson_file(latitude, longitude, filename="bbox.geojson"):
    geojson_data = generate_bbox_geojson(latitude, longitude)
    
    with open(filename, "w") as file:
        file.write(geojson_data)

# # Example usage
# if __name__ == "__main__":
#     latitude = 48.1351  # Example latitude
#     longitude = 11.5820  # Example longitude
#     print(generate_bbox_geojson(latitude, longitude))


In [8]:
latitude = 38.4425  # Example latitude
longitude = -0.2857  # Example longitude
create_geojson_file(latitude, longitude, "bounding_box.geojson")